In [1]:
import pandas as pd

def work(df: pd.DataFrame):    
    res = df.copy()
    for i in df.columns:
        if i != 'SalePrice' and i != 'Id':
            if df[i].dtype == 'object' or i == 'MSSubClass':
                res = pd.concat([res, pd.get_dummies(res[i], prefix=i)], axis='columns').drop(i, axis='columns')
            else:
                res[i] = (res[i] - res[i].mean()) / (res[i].max() - res[i].min())
                res[i] = res[i].fillna(res[i].mean())
    return res

In [2]:
train = pd.read_csv('input/train.csv')
train_n = len(train)
test = pd.read_csv('input/test.csv')

data = pd.concat([train, test])
data = work(data)

train = data[:train_n]
test = data[train_n:]

input_shape = [len(train.columns) - 2]  # 减去 SalePrice 和 Id

print(f'''input_shape: {input_shape}
train_n: {len(train)}''')

input_shape: [303]
train_n: 1460


In [3]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=input_shape),
    layers.Dropout(rate=0.3),
    layers.Dense(512, activation='relu'),
    layers.Dropout(rate=0.3),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(rate=0.3),
    layers.Dense(64, activation='relu'),
    # layers.Dropout(rate=0.3),
    layers.Dense(32, activation='relu'),
    # layers.Dropout(rate=0.3),
    layers.Dense(16, activation='relu'),
    # layers.Dropout(rate=0.3),
    layers.Dense(8, activation='relu'),
    layers.Dense(4, activation='relu'),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='relu'),
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)

history = model.fit(
    x=train.drop(['Id', 'SalePrice'], axis='columns'),
    y=train.SalePrice,
    batch_size=256,
    validation_split=0.9,
    epochs=100,
    verbose=0,
)
history_df = pd.DataFrame(history.history)
history_df.loc[:].plot()

2023-02-21 11:53:47.042519: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 11:53:47.046576: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


KeyboardInterrupt: 

In [ ]:
result = pd.Series([x[0] for x in model.predict(test.drop(['Id', 'SalePrice'], axis='columns'))], index=test.Id, name='SalePrice')
result

46/46 [==============================] - 0s 5ms/step


Id
1461    887.515869
1462    899.926514
1463    985.954346
1464    964.525146
1465    907.021301
           ...    
2915    773.192810
2916    874.464172
2917    960.313354
2918    845.519165
2919    983.643982
Name: SalePrice, Length: 1459, dtype: float32

In [ ]:
result.to_csv('submission.csv')

In [8]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

2.9.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
